In [1]:
import pandas as pd
import numpy as np 
import datetime
import math
import gc
gc.collect()

from tqdm import tqdm

# Load Data

In [2]:
years = list(range(2002, int(datetime.datetime.now().year)))

df = pd.DataFrame()
for year in tqdm(years):
    path = './output/'+str(year)+'/'+str(year)+'_pbp.csv'
    sea_df = pd.read_csv(path)
    df = pd.concat([df,sea_df])

num_plays = len(df)
print(str(num_plays) + " plays were loaded")

100%|██████████| 17/17 [00:21<00:00,  1.86s/it]

2223578 plays were loaded


In [3]:
print(list(df))
print(list(df.play_type.unique()))

['away', 'clock.minutes', 'clock.seconds', 'defense', 'defense_conference', 'defense_score', 'distance', 'down', 'drive_id', 'home', 'id', 'offense', 'offense_conference', 'offense_score', 'period', 'play_text', 'play_type', 'yard_line', 'yards_gained', 'game_id']
['Uncategorized', 'Kickoff Return (Offense)', 'Sack', 'Rush', 'Punt Return', 'Penalty', 'Pass Completion', 'Pass Incompletion', 'Safety', 'End Period', 'Pass Interception', 'Blocked Punt Touchdown', 'Fumble Recovery (Own)', 'Timeout', 'Fumble Recovery (Opponent)', 'Two Point Pass', 'Two Point Rush', 'Interception Return Touchdown', 'Blocked Punt', 'Punt Return Touchdown', 'Blocked Field Goal', 'Kickoff Return (Defense)', 'Fumble Return Touchdown', 'Kickoff Return Touchdown', 'Blocked PAT', 'Blocked Field Goal Touchdown', 'Missed Field Goal Return Touchdown', 'Punt', 'Pass', 'Kickoff', 'Extra Point Good', 'Field Goal Good', 'Field Goal Missed', 'Extra Point Missed', '2pt Conversion', 'Offensive 1pt Safety', 'Pass Reception', '

In [4]:
def fix_uncat(play_type, play_text):
    if play_type != 'Uncategorized':
        return play_type
    else:
        if isinstance(play_text,str):
            if "Start of the 1st quarter." in play_text:
                return "End Period"
            elif "Start of the 2nd quarter." in play_text:
                return "End Period"
            elif "Start of the 3rd quarter." in play_text:
                return "End of Half"
            elif "Start of the 4th quarter." in play_text:
                return "End Period"
            elif "Start of overtime." in play_text:
                return "End Period"
            elif "End of the game." in play_text:
                return "End of Game"
            elif "Extra point" in play_text:
                if "is good" in play_text:
                    return "Extra Point Good"
                elif "is no good." in play_text[-13:]:
                    return "Extra Point Missed"
                else:
                    return play_type
            elif "field goal" in play_text:
                if "is good" in play_text:
                    return "Field Goal Good"
                elif "is no good." in play_text[-13:]:
                    return "Field Goal Missed"
                else:
                    print(play_text)
                    return play_type
            elif "missed PAT returned." in play_text:
                return "Extra Point Missed"
            else:
                return play_type
    return play_type

df['play_type'] = df.apply(lambda row: fix_uncat(row['play_type'], row['play_text']),axis=1)

uncat = df.loc[df.play_type=='Uncategorized']
print(len(uncat))
print(uncat.play_text.head(50))

119
3940     DeAngelo Hall (VT) took lateral and rushed for...
11077    Terrence Biggers (MSU) took lateral and rushed...
11092    Derek Abney (UK) took lateral and rushed for 1...
13313    Terrance Phillips (PSU) took lateral and rushe...
28227    Andrico Hines (MTSU) took lateral and rushed f...
30739    Lance Pendleton (BYU) took lateral and rushed ...
31087    Chris Bruhn (WSU) took lateral and rushed for ...
33853    Tim Blackwell (USM) took lateral and rushed fo...
36684    Bruce Gradkowski (TOL) took lateral and rushed...
39523    Michael Turner (NIU) took lateral and rushed f...
43241    Jason Samples (TSU) took lateral and rushed fo...
48186    Steve Breaston (MICH) took lateral and rushed ...
52925    Duane Coleman (CLEM) took lateral and rushed f...
53764    Scott Lunde (WSU) took lateral and rushed for ...
63806    Aric Williams (OSU) took lateral and rushed fo...
75692    Garrett Lepisto (UCLA) took lateral and rushed...
83122    Sean Taylor (MIA) took lateral and rushed f

# Need Separate Model for XP, Kickoffs

In [5]:
# drop_cols
separate = ['End Period', 'Kickoff Return (Offense)', 'Extra Point Good', 'Timeout',
 'End of Half', 'End of Game', 'Two Point Pass', 'Two Point Rush', 
 'Kickoff Return (Defense)', 'Uncategorized', 'Kickoff Return Touchdown', 'Blocked PAT','Kickoff', 
 'Extra Point Missed', '2pt Conversion', 'Defensive 2pt Conversion', 'Offensive 1pt Safety']

print(len(df))
sep_df = df.loc[df.play_type.isin(separate)]
print(len(sep_df))
df = df.loc[~df.play_type.isin(separate)]
print(len(df))


2223578
272728
1950850


In [6]:
# drop overtime and 61 period 0 entries
print(len(df))
df = df.loc[df.period.isin([1,2,3,4])]
print(len(df))

1950850
1944000


In [7]:
# instead of zero its nan for clock.seconds and clock.minutes
df['clock.seconds'] = df['clock.seconds'].fillna(0)
df['clock.minutes'] = df['clock.minutes'].fillna(0)

print(list(df['clock.seconds'].unique()))

gb = df.groupby(['clock.seconds'])['id'].count()
gb = gb.sort_values()
print(gb.head(70))

[0.0, 25.0, 45.0, 30.0, 11.0, 15.0, 40.0, 55.0, 18.0, 22.0, 54.0, 23.0, 33.0, 44.0, 20.0, 34.0, 4.0, 10.0, 53.0, 56.0, 51.0, 21.0, 6.0, 16.0, 46.0, 3.0, 58.0, 7.0, 47.0, 27.0, 57.0, 17.0, 48.0, 37.0, 24.0, 14.0, 50.0, 5.0, 35.0, 43.0, 39.0, 52.0, 26.0, 36.0, 42.0, 12.0, 2.0, 32.0, 28.0, 8.0, 31.0, 19.0, 9.0, 29.0, 13.0, 41.0, 59.0, 38.0, 49.0, 1.0]
clock.seconds
59.0     23264
39.0     23604
29.0     23654
19.0     24132
26.0     24201
57.0     24310
31.0     24386
6.0      24445
37.0     24556
36.0     24586
3.0      24733
9.0      24748
34.0     24807
4.0      24893
58.0     24915
14.0     24915
27.0     24955
24.0     24977
32.0     24984
16.0     25026
43.0     25026
17.0     25185
46.0     25213
41.0     25230
13.0     25291
1.0      25308
28.0     25311
21.0     25325
49.0     25463
42.0     25633
23.0     25657
38.0     25704
7.0      25739
2.0      25808
47.0     25844
11.0     26020
8.0      26148
51.0     26210
18.0     26332
56.0     26556
48.0     26706
12.0     26760
33.0 

In [8]:
# # calculate time remaining in half
def tr_half(period, minutes, seconds):
    tr = 0
    if period in [1,3]:
        # add a quarter of time remaining
        tr += 900
    tr += (60 * minutes + seconds)
    return tr

def tr_game(period, minutes, seconds):
    quarters_left = 4-period
    added_secs = 15*60*quarters_left
    return (60*minutes + seconds + added_secs)

df['tr_half'] = df.apply(lambda row: tr_half(row['period'],row['clock.minutes'],row['clock.seconds']),axis=1)
df['tr_game'] = df.apply(lambda row: tr_game(row['period'],row['clock.minutes'],row['clock.seconds']),axis=1)

print(df[['period','clock.minutes','clock.seconds','tr_half','tr_game']].head(10))

    period  clock.minutes  clock.seconds  tr_half  tr_game
2        1           14.0            0.0   1740.0   3540.0
3        1           14.0           25.0   1765.0   3565.0
4        1           14.0           45.0   1785.0   3585.0
5        1           13.0           30.0   1710.0   3510.0
6        1           11.0           11.0   1571.0   3371.0
7        1           11.0            0.0   1560.0   3360.0
8        1           13.0           15.0   1695.0   3495.0
9        1           12.0           45.0   1665.0   3465.0
10       1           12.0           25.0   1645.0   3445.0
11       1           12.0            0.0   1620.0   3420.0


In [9]:
# drop clock numbers, not needed anymore 
df = df.drop(columns=['clock.minutes','clock.seconds'])

# Get Desired Features

Need 6 variables. Well 8.

Down  
Seconds left in half  
Yards to go for touchdown (log?)  
Yards to go for first down (log?)  
Goal to goal indicator  
Under 2 minutes indicator  

Using two others to weigh observations
-number of drives to next score
-absolute score differential

Also need target variable. Next Score.


In [10]:
print(len(df))
df = df.dropna(subset=['play_text'])
print(len(df))

1944000
1943642


In [11]:
def add_tds(play_type, play_text):
    if play_type != 'Penalty':
        if isinstance(play_text, str):
            if 'Touchdown' in play_type:
                return 1
            elif 'touchdown' in play_text:
                return 1
            elif 'for a TD' in play_text:
                return 1
    return 0
    
df['touchdown'] = df.apply(lambda row: add_tds(row['play_type'],row['play_text']), axis=1)
    


In [12]:
td_plays = df.loc[df['touchdown']==1]
print(list(td_plays.play_type.unique()))

['Pass Completion', 'Blocked Punt Touchdown', 'Rush', 'Fumble Recovery (Own)', 'Interception Return Touchdown', 'Punt Return Touchdown', 'Fumble Return Touchdown', 'Blocked Field Goal Touchdown', 'Missed Field Goal Return Touchdown', 'Sack', 'Pass Incompletion', 'Passing Touchdown', 'Rushing Touchdown', 'Punt', 'Fumble Recovery (Opponent)', 'Pass Reception', 'Blocked Punt', 'Pass Interception Return', 'Blocked Field Goal']


In [13]:
defensive_tds = ['Blocked Punt Touchdown', 'Interception Return Touchdown','Punt Return Touchdown',
            'Fumble Return Touchdown','Blocked Field Goal Touchdown','Missed Field Goal Return Touchdown',
             'Sack']

not_touchdowns = ['Pass Incompletion']

offensive_tds = ['Pass Completion','Rush','Fumble Recovery (Own)','Rushing Touchdown','Passing Touchdown']

# split into offensive and defensive touchdowns

def cat_offense(play_type, touchdown):
    if touchdown == 1:
        if play_type not in defensive_tds:
            if play_type != 'Pass Incompletion':
                return 1
    return 0

def cat_defense(play_type, touchdown):
    if touchdown ==1:
        if play_type in defensive_tds:
            return 1
    return 0

df['offensive_TD'] = df.apply(lambda row: cat_offense(row['play_type'],row['touchdown']),axis=1)

df['defensive_TD'] = df.apply(lambda row: cat_defense(row['play_type'],row['touchdown']),axis=1)

print(list(df))



['away', 'defense', 'defense_conference', 'defense_score', 'distance', 'down', 'drive_id', 'home', 'id', 'offense', 'offense_conference', 'offense_score', 'period', 'play_text', 'play_type', 'yard_line', 'yards_gained', 'game_id', 'tr_half', 'tr_game', 'touchdown', 'offensive_TD', 'defensive_TD']


In [14]:
# add field goals and safeties
def add_fgs(play_type):
    if play_type == 'Field Goal Good':
        return 1
    return 0

def add_safeties(play_type):
    if play_type == 'Safety':
        return 1
    return 0

df['fg'] = df.apply(lambda row: add_fgs(row['play_type']),axis=1)

df['safety'] = df.apply(lambda row: add_safeties(row['play_type']),axis=1)


# weird = non_tds.loc[non_tds['play_type'] == 'Interception Return Touchdown']
# for w in list(weird.play_text.values):
#     print(w)

In [15]:
# add score for each drive
drive_gb = df.groupby(['drive_id'])['offensive_TD','defensive_TD','fg','safety'].max().reset_index()

drive_gb['drive_score'] = 7 * drive_gb['offensive_TD'] + -7 * drive_gb['defensive_TD'] + 3 * drive_gb['fg'] + -2 * drive_gb['safety']
drive_gb['drive_score'] = drive_gb['drive_score'].astype(int)

possible = [0, 3, 7, -2, -7]

not_possible = drive_gb.loc[~drive_gb['drive_score'].isin(possible)]
print(not_possible)
print(len(not_possible))

drive_ids = list(df.drive_id.unique())
# print(len(drive_ids))

           drive_id  offensive_TD  defensive_TD  fg  safety  drive_score
37852    4010320813             1             0   1       0           10
199623  32266005905             0             0   1       1            1
241117  40054786811             1             0   1       0           10
3


In [16]:
## 32266005905 remove safety, it was on kickoff somehow
df = df.loc[df['id']!=322660059036]

## 4010320813 has two plays from 4010320812
df.loc[df['id'] == 401032081101874002, ['drive_id']] = 4010320812
df.loc[df['id'] == 401032081101907203, ['drive_id']] = 4010320812

## 40054786811 has two drives
df.loc[(df['drive_id']==40054786811) & (df['offense']=='Baylor'), ['drive_id']] = 4005478681100

In [17]:
# since clock numbers aren't consistent for some plays, I am sorting drives by average time remaining 
# of all plays on the drive

tr = df.groupby(['drive_id'])['tr_game'].mean().reset_index()

tr = tr.rename(columns={'tr_game':'avg_drive_time'})
print(tr.head(5))

df = pd.merge(left=df, right=tr, how='left', on=['drive_id','drive_id'])

     drive_id  avg_drive_time
0  4005476401         3518.75
1  4005476402         3361.00
2  4005476403         3255.25
3  4005476404         3145.60
4  4005476405         3064.50


In [38]:
def get_next(current_drive, current_offense, scoring_drives):
    scoring_drives = scoring_drives.sort_values(by='drive_no',ascending=True)
    for index,row in scoring_drives:
        if row['drive_no'] >= current_drive:
            if row['offense'] == current_offense:
                return (row['drive_no'] - current_drive), (row['drive_score'])
            else:
                return (row['drive_no'] - current_drive), (-1 * row['drive_score'])
    return 0,0

cd = 6
co = 'Kentucky'

scoring_drives_1H = pd.DataFrame([[3,'Kentucky',3],[5,'Louisville',7],[7,'Kentucky',3]],columns=['drive_no','offense','drive_score'])

print(scoring_drives_1H)
dtns = get_next(cd, co, scoring_drives_1H)
print(dtns)

   drive_no     offense  drive_score
0         3    Kentucky            3
1         5  Louisville            7
2         7    Kentucky            3


ValueError: too many values to unpack (expected 2)

In [21]:
games=df.groupby(['game_id'])

new_df = []

counter = 0
for game, game_plays in games:
    counter+=1
    if counter % 500 == 0:
        print(counter)
    # sort by time remaining to order them
    ordered = game_plays.sort_values(by=['avg_drive_time'],ascending=False)
    # label drive numbers
    i = ordered.drive_id
    ordered['drive_no'] = i.ne(i.shift()).cumsum()
    
    ordered['is_scoring_drive'] = ordered['drive_score'].where(s != 0, 1, 0)
    ordered['half'] = ordered['period'].where(s < 3, 1, 2)
    
    scoring_drives_1H = ordered.loc[(ordered['is_scoring_drive']==1)&(ordered['half']==1)]
    scoring_drives_2H = ordered.loc[(ordered['is_scoring_drive']==1)&(ordered['half']==2)]
    
    sd_next_1H = scoring_drives_1H[['drive_no','offense','drive_score']]
    sd_next_2H = scoring_drives_2H[['drive_no','offense','drive_score']]
    
    for index, row in ordered.iterrows():
        
    print(ordered.head())
    
    new_df = pd.concat([new_df,ordered],ignore_index=True)

10
(1595, 27)
20
(3316, 27)
30
(4970, 27)
40
(6607, 27)
50
(8291, 27)
60
(9945, 27)
70
(11608, 27)
80
(13284, 27)
90
(14956, 27)
100
(16653, 27)
110
(18338, 27)
120
(20033, 27)
130
(21714, 27)
140
(23335, 27)
150
(24988, 27)
160
(26715, 27)
170
(28352, 27)
180
(30075, 27)
190
(31761, 27)
200
(33451, 27)
210
(35144, 27)
220
(36646, 27)
230
(38350, 27)
240
(39961, 27)
250
(41658, 27)
260
(43234, 27)
270
(44839, 27)
280
(46466, 27)
290
(48132, 27)
300
(49920, 27)
310
(51526, 27)
320
(53288, 27)
330
(54910, 27)


KeyboardInterrupt: 

In [ ]:
print(len(new_df))
print(len(df))
    

In [ ]:
games=df.groupby(['game_id'])

new_df = pd.DataFrame()

counter = 0
for game, game_plays in games:
    counter+=1
    if counter % 1000 == 0:
        print(counter)
    # sort by time remaining to order them
    ordered = game_plays.sort_values(by=['avg_drive_time'],ascending=False)
    # label drive numbers
    i = ordered.drive_id
    ordered['drive_no'] = i.ne(i.shift()).cumsum()
    
    
    new_df = pd.concat([new_df,ordered])